In [21]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, Layout
from IPython.display import Markdown, display, HTML
from IPython import utils
mpl.style.use('seaborn')
from sklearn.ensemble import RandomForestClassifier

In [22]:
df = pd.read_csv('mexico_clean_data.csv')
df.set_index('ID_REGISTRO', inplace=True)
print(df.shape)
df.head(100)

(1044936, 16)


,SEXO,ENTIDAD_NAC,INTUBADO,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,Age Ranges
ID_REGISTRO,,,,,,,,,,,,,,,,
z482b8,2,9,2,2,41,2,2,2,2,2,2,2,2,2,2,41 - 55
z49a69,1,23,1,1,66,2,1,2,2,2,1,2,1,2,2,56 - 70
z23d9d,2,24,2,2,29,2,2,2,2,2,2,2,98,2,2,25 - 40
zz8e77,2,9,2,2,34,2,2,1,1,2,2,2,2,2,2,25 - 40
z1b0d1,1,1,2,2,48,2,1,2,2,2,2,2,1,2,2,41 - 55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
z5ac3a,1,31,2,2,31,2,2,2,1,2,2,2,2,2,2,25 - 40
z1d478,1,29,2,2,24,2,2,2,2,2,2,2,2,2,2,<25
z479a5,1,31,2,2,8,2,2,2,2,2,2,2,2,2,2,<25


In [23]:
df=df.reset_index()

In [24]:
df

,ID_REGISTRO,SEXO,ENTIDAD_NAC,INTUBADO,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,Age Ranges
0,z482b8,2,9,2,2,41,2,2,2,2,2,2,2,2,2,2,41 - 55
1,z49a69,1,23,1,1,66,2,1,2,2,2,1,2,1,2,2,56 - 70
2,z23d9d,2,24,2,2,29,2,2,2,2,2,2,2,98,2,2,25 - 40
3,zz8e77,2,9,2,2,34,2,2,1,1,2,2,2,2,2,2,25 - 40
4,z1b0d1,1,1,2,2,48,2,1,2,2,2,2,2,1,2,2,41 - 55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044931,152a09,1,28,2,2,17,2,2,2,2,2,2,2,2,2,2,<25
1044932,0ebaaf,1,16,2,2,57,2,2,2,2,2,2,2,2,2,2,56 - 70
1044933,05c07c,2,26,2,2,35,2,2,2,2,2,2,2,2,2,2,25 - 40
1044934,1de190,1,9,2,2,38,2,2,2,2,2,2,2,2,2,2,25 - 40


In [25]:
df=df.drop(['ID_REGISTRO'], axis=1)

In [26]:
si_no = {
    1: 1,
    2: 0,
    97: None,
    98: None,
    99: None
}
conversions= {
    'SEXO': {
        1: 'MUJER',
        2: 'HOMBRE',
        3: 'NA'
    }, 
    'INTUBADO': si_no,
    'NEUMONIA': si_no,
    'EMBARAZO': si_no,
    'DIABETES': si_no,
    'EPOC': si_no,
    'ASMA': si_no,
    'INMUSUPR': si_no,
    'HIPERTENSION': si_no,
    'CARDIOVASCULAR': si_no,
    'OBESIDAD': si_no,
    'RENAL_CRONICA': si_no,
    'TABAQUISMO': si_no
}

dt = df.copy()
for k,v in conversions.items():
    dt[k] = dt[k].replace(v)
    
dt = dt.drop(columns=['EDAD', 'ENTIDAD_NAC'])
dt.head()

,SEXO,INTUBADO,NEUMONIA,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,Age Ranges
0,HOMBRE,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41 - 55
1,MUJER,1,1,0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,56 - 70
2,HOMBRE,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,25 - 40
3,HOMBRE,0,0,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,25 - 40
4,MUJER,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,41 - 55


In [27]:
# Remove NaNs. Will lose less than 3% of the data
dt = dt.dropna()
dt.head()
# Will remove nans for now to avoid unnecessary complexity on the model. Could consider including them in the future

,SEXO,INTUBADO,NEUMONIA,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,Age Ranges
0,HOMBRE,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41 - 55
1,MUJER,1,1,0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,56 - 70
3,HOMBRE,0,0,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,25 - 40
4,MUJER,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,41 - 55
5,MUJER,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,56 - 70


In [28]:
# Convert columns to binary
for col in dt:
    if len(dt[col].unique()) == 2 and dt[col].dtype != object:
        dt[col] = dt[col].astype(np.int0)
cols = ['INTUBADO','NEUMONIA','EMBARAZO']
for col in cols:
    dt[col] = dt[col].astype(np.int0)

In [29]:
# Encode categorical features
dd = pd.get_dummies(dt)
dd.head()

,INTUBADO,NEUMONIA,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,SEXO_HOMBRE,SEXO_MUJER,Age Ranges_25 - 40,Age Ranges_41 - 55,Age Ranges_56 - 70,Age Ranges_71 - 85,Age Ranges_85+,Age Ranges_<25
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
1,1,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0
3,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0
5,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0


In [30]:
# Check dataset balance
display(dd.groupby('INTUBADO').size(),
dd.groupby('NEUMONIA').size())

INTUBADO
0    982113
1     34549
dtype: int64

NEUMONIA
0    872309
1    144353
dtype: int64

In [31]:
target = dd["INTUBADO"]
target_names = ["negative", "positive"]

In [32]:
data = dd.drop(["INTUBADO","NEUMONIA"], axis=1)
feature_names = data.columns
data.head()

,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,SEXO_HOMBRE,SEXO_MUJER,Age Ranges_25 - 40,Age Ranges_41 - 55,Age Ranges_56 - 70,Age Ranges_71 - 85,Age Ranges_85+,Age Ranges_<25
0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0
3,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0


In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [34]:
# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=200, n_jobs=-1)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.966144173492914

In [35]:
sorted(zip(rf.feature_importances_, feature_names),reverse=True)

[(0.17313992819261217, 'Age Ranges_71 - 85'),
 (0.1568926244587719, 'Age Ranges_56 - 70'),
 (0.13938764347867552, 'DIABETES'),
 (0.09499054140030502, 'HIPERTENSION'),
 (0.06533552161239173, 'Age Ranges_25 - 40'),
 (0.04455844698348807, 'RENAL_CRONICA'),
 (0.04273492849541819, 'Age Ranges_41 - 55'),
 (0.04219247945750837, 'OBESIDAD'),
 (0.03352026420462369, 'INMUSUPR'),
 (0.0313621550164075, 'CARDIOVASCULAR'),
 (0.03042905524673184, 'TABAQUISMO'),
 (0.03010554790682954, 'EPOC'),
 (0.027061403304127346, 'SEXO_HOMBRE'),
 (0.026367520076111754, 'SEXO_MUJER'),
 (0.02610430458461034, 'Age Ranges_<25'),
 (0.02066956695667246, 'ASMA'),
 (0.012864668827133573, 'Age Ranges_85+'),
 (0.00228339979758083, 'EMBARAZO')]

In [36]:
from sklearn.metrics import classification_report, confusion_matrix

In [37]:
dd['INTUBADO'].value_counts()

0    982113
1     34549
Name: INTUBADO, dtype: int64

In [38]:
dd.shape

(1016662, 20)

In [39]:
df_prueba=dd.query("INTUBADO==1").sample(34500)

In [40]:
df_prueba1=dd.query("INTUBADO==0").sample(34500)

In [41]:
df_prueba=df_prueba.append(df_prueba1)

In [42]:
df_prueba['INTUBADO'].value_counts()

1    34500
0    34500
Name: INTUBADO, dtype: int64

In [43]:
data_input=df_prueba.drop(["INTUBADO","NEUMONIA"], axis=1)

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_input, df_prueba['INTUBADO'], random_state=42, stratify=df_prueba['INTUBADO'])

In [45]:
# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.744

In [46]:
# Calculate the mean_squared_error and the r-squared value
# for the testing data

from sklearn.metrics import mean_squared_error, r2_score

### BEGIN SOLUTION

# Use our model to make predictions
predicted = rf.predict(X_test)

# Score the predictions with mse and r2
mse = mean_squared_error(y_test, predicted)
r2 = r2_score(y_test, predicted)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

### END SOLUTION

Mean Squared Error (MSE): 0.256
R-squared (R2 ): -0.02400000000000002


In [47]:
confusion_matrix(y_test, predicted)

array([[6221, 2404],
       [2012, 6613]], dtype=int64)

In [48]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.76      0.72      0.74      8625
           1       0.73      0.77      0.75      8625

    accuracy                           0.74     17250
   macro avg       0.74      0.74      0.74     17250
weighted avg       0.74      0.74      0.74     17250



In [49]:
import pickle

In [50]:
filename='intubation_pred.pkl'
pickle.dump(rf,open(filename,'wb'))

In [51]:
!pip install flask-cors

You should consider upgrading via the 'python -m pip install --upgrade pip' command.
